In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.parser import TextDocPreprocessor2
doc_preprocessor = TextDocPreprocessor2('data2')

In [3]:
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser


n_docs=5

corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)

Clearing existing...
Running UDF...
[========                                ] 20%data2/doc7
[========                                ] 20%data2/doc40
[================                        ] 40%data2/doc116
[========================                ] 60%data2/doc137
[================================        ] 80%data2/doc19
[========================================] 100%data2/doc20
[================================================] 120%data2/doc71
data2/doc14
data2/doc95
data2/doc59
data2/doc58
data2/doc3
data2/doc141
data2/doc56
data2/doc104
data2/doc13
data2/doc72
data2/doc129
data2/doc9
data2/doc78
data2/doc106
data2/doc93
data2/doc131
data2/doc51
data2/doc8
data2/doc94
data2/doc47
data2/doc23
data2/doc144
data2/doc67
data2/doc74
data2/doc44
data2/doc70
data2/doc64
data2/doc133
data2/doc124
data2/doc118
data2/doc111
data2/doc100
data2/doc82
data2/doc114
data2/doc123
data2/doc69
data2/doc4
data2/doc103
data2/doc12
data2/doc80
data2/doc149
data2/doc21
data2/doc61
data2/doc79
data2/do

In [4]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())


('Documents:', 151)
('Sentences:', 1609)


In [5]:
from snorkel.models import candidate_subclass

Disaster = candidate_subclass('Disaster', ['Name','Location'])

In [6]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DateMatcher, LocationMatcher
ngrams         = Ngrams(n_max=7)
#person_matcher = PersonMatcher(longest_match_only=True)
location_matcher = LocationMatcher(longest_match_only=True)
date_matcher = DateMatcher(longest_match_only=True)
#dict=['earthquake','flood','cyclone','fire']
#dictionary_matcher=DictionaryMatch(d=dict)
cand_extractor = CandidateExtractor(Disaster, [ngrams, ngrams], [date_matcher, location_matcher])

In [7]:
from snorkel.models import Document
#from util import number_of_people


docs = session.query(Document)

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 3 == 2:
            dev_sents.add(s)
        elif i % 3 == 1:
            train_sents.add(s)
        else:
            train_sents.add(s)

In [8]:
print(len(dev_sents),len(train_sents),len(test_sents))

(531, 1078, 0)


In [9]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(Disaster).filter(Disaster.split == i).count())



Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 68)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 21)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 0)
CPU times: user 13.6 s, sys: 908 ms, total: 14.5 s
Wall time: 14 s


In [10]:
import re
from snorkel.lf_helpers import (
    get_between_tokens,
    get_text_between, get_tagged_text,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

                   
meteorological_disasters = { 'storm', 'bizzard', 'tornado', 'hurricane', 'cyclone', 'avalanches', 'heatwave','coldwave' 'drought', 'hailstorm'}
hydrological_disasters = { 'flood' , 'tsunami', 'limnic eruption' , 'famine' , 'forest fire' }

geological_disasters = { 'earthquake', 'volcano' , 'rock fall'}

                   
biological_disasters = { 'epidemic', 'pandemic' , 'influenza'}

manmade_disasters = { 'blast', 'bomb explosion', 'bomb blast', 'bomb', 'bombings', 'surgical strike', 'shooting', 'firing', 'gun fire' , 'riots' , 'armed conflict'}

def LF_mdisaster(c):
     return 1 if len(meteorological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_hdisaster(c):
    return 1 if len(hydrological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_gdisaster(c):
    return 1 if len(geological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_bdisaster(c):
    return 1 if len(biological_disasters.intersection(get_between_tokens(c))) > 0 else 0

def LF_mandisaster(c):
    return 1 if len(manmade_disasters.intersection(get_between_tokens(c))) > 0 else 0
def LF_tdisaster(c):
      return 1 if 'train’' in get_between_tokens(c) and 'collided’' in get_right_tokens(c) else 0
def LF_killed(c):
    return 1 if re.search (r'{{A}}+\w+\s+killed+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I)  else 0

def LF_injured(c):
    return 1 if re.search (r'{{A}}+\w+\s+injured+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I)  else 0

def LF_dead(c):
    return 1 if re.search (r'{{A}}+\w+\s+dead+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_damage(c):
    return 1 if re.search (r'{{A}}+\w+\s+heavy+\s+damage+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_conflict(c):
    return 1 if re.search (r'{{A}}+\w+\s+conflict+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0


def ltp(x):
    return '(' + '|'.join(x) + ')'

trigger1 = ['killed', 'kills', 'killing']

def LF_kill(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger1) + '.\w', 1)
trigger2 = ['violence', 'violent', 'rioting']
def LF_violence(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger2) + '.\w', 1)
trigger3 = ['crashed’,’collided’,’hit’']

def LF_crash(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger3) + '.\w', 1)

trigger4 = ['surgical strike', 'strike', 'struck', 'hit the target']

def LF_strike(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger4) + '.\w', 1)

trigger5 = ['cold', 'heat']

def LF_wave(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger4) +"wave"+ '.\w', 1)
def LF_cold(c):
    return 1 if re.search (r'{{A}}+\w+\s+cold+arctic +air+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

trigger6 = ['flood' ,'floods' , 'flooding', 'flooded']
def LF_flood(c):
     return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger4) + '.\w', 1)

trigger7 = ['eruption' ,'erupting' , 'erupted',  'erupt']

def LF_eruption(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger7) + '.\w', 1)

trigger8 = ['fell off' ,'slide', 'fall']

#def LF_rock(c):
    #return rule_regex_search_btw_AB(c, '.\w' + rock + ltp(trigger8) + '.\w', 1)

trigger9 = ['cholera' ,'ebola', 'dengue']

def LF_epidemic(c):
    return rule_regex_search_btw_AB(c, '.\w' + ltp(trigger9) + '.\w', 1)


def LF_tsunami(c):
    return 1 if re.search (r'{{A}}+\w+\s+tsunami+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_confront(c):
    return 1 if re.search (r'{{A}}+\w+\s+armed+confrontation+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_rammed(c):
    return 1 if re.search (r'{{A}}+LF_rammed\w+\s+rammed+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_smashed(c):
    return 1 if re.search (r'{{A}}+\w+\s+smashed+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_exploded(c):
    return 1 if re.search (r'{{A}}+\w+\s+exploded+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_shipwrecked(c):
    return 1 if re.search (r'{{A}}+\w+\s+shipwrecked+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_nuclearmeltdown(c):
    return 1 if re.search (r'{{A}}+\w+\s+nuclear meltdown.+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_tornado(c):
    return 1 if re.search (r'{{A}}+\w+\s+tornado(es)+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_limniceruption(c):
    return 1 if re.search (r'{{A}}+\w+\s+limnic eruption+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

def LF_wildfire(c):
    return 1 if re.search (r'{{A}}+\w+\s+wildfire(es)+\s+\w+{{B}}',  get_tagged_text(c), flags =re.I) else 0

In [11]:
LFs = [
     LF_mdisaster,  LF_gdisaster,  LF_hdisaster,  LF_bdisaster,  LF_mandisaster, LF_tdisaster, LF_wildfire,LF_limniceruption, LF_tornado
,LF_nuclearmeltdown, LF_shipwrecked  ,LF_exploded, LF_smashed, LF_rammed,  LF_confront,LF_tsunami ,LF_epidemic,LF_eruption,LF_flood,
LF_wave,LF_strike, LF_crash, LF_violence, LF_kill, LF_conflict, LF_damage, LF_dead, LF_injured, LF_kill ]

In [12]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

In [13]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

np.random.seed(1701)
%time L_train = labeler.apply(split=0)
L_train

In [14]:
f=open("output1.txt",'w')
f.seek(0)
f.write(str(L_train))
f.close()

In [16]:
L_train.get_candidate(session, 0)

Disaster(Span("5100", sentence=1563, chars=[172,175], words=[31,31]), Span("the Lower 9th Ward", sentence=1563, chars=[207,224], words=[38,41]))

In [18]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)



In [19]:
gen_model.weights.lf_accuracy

array([ 0.12413886,  0.13380363,  0.10580794,  0.07506928,  0.08489036,
        0.06854687,  0.09529703,  0.0820241 ,  0.08241606,  0.04963128,
        0.08174002,  0.07691405,  0.06367795,  0.08603358,  0.04208157,
        0.0800324 ,  0.08263257,  0.09499151,  0.08173692,  0.07733761,
        0.07974053,  0.07920366,  0.07611276,  0.09176501,  0.08364014,
        0.06999395,  0.06070118,  0.0937695 ])

In [20]:
gen_model.learned_lf_stats()

,Accuracy,Coverage,Precision,Recall
0,0.555042,0.6713,0.558449,0.367917
1,0.567604,0.6723,0.571344,0.384278
2,0.547968,0.6619,0.547009,0.357542
3,0.547190,0.6601,0.541962,0.365922
4,0.531814,0.6758,0.533896,0.353551
5,0.532389,0.6638,0.531446,0.355746
6,0.541295,0.6623,0.546993,0.361133
7,0.534496,0.6653,0.535207,0.353352
8,0.547238,0.6626,0.543340,0.358939
9,0.522184,0.6649,0.520311,0.347566


In [21]:
train_marginals = gen_model.marginals(L_train)

In [22]:
f=open("train_marginals1.txt",'w')
f.write(str(train_marginals))
f.close()

In [23]:
L_dev = labeler.apply(split=1)


In [24]:
f=open("L_dev.txt1",'w')
f.write(str(L_dev))
f.close()

In [25]:
L_dev.get_candidate(session, 0)

Disaster(Span("Sunday", sentence=1706, chars=[3,8], words=[1,1]), Span("Southwest", sentence=1706, chars=[14,22], words=[3,3]))

In [26]:
L_train.get_candidate(session, 1)

Disaster(Span("20", sentence=858, chars=[131,132], words=[28,28]), Span("Ramanathapuram", sentence=858, chars=[51,64], words=[11,11]))

In [27]:
L_train.get_candidate(session, 2)

Disaster(Span("20", sentence=858, chars=[92,93], words=[18,18]), Span("Ramanathapuram", sentence=858, chars=[51,64], words=[11,11]))

In [28]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

In [29]:
f=open("L_gold.txt1",'w')
f.write(str(L_gold_dev))
f.close()

In [34]:
def predict_labels(marginals):
    predicted_labels=[]
    for i in marginals:
        if(i<=0.5): 
            predicted_labels.append(-1)
        else:
            predicted_labels.append(1)
    return predicted_labels
x=predict_labels(train_marginals)
f=open("L_pred.txt1",'w')
f.write(str(x))
#f.write(str(len(x))
f.close()

In [35]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
x=np.array(x)
list=np.ones((68,), dtype=int)
precision_recall_fscore_support(x, list, average='macro')

(0.11029411764705882, 0.5, 0.18072289156626503, None)